In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 1
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取1秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_3s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_3s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 26.25 seconds
Progress: 100% - Completed
Data Loading Time: 26.25 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_3S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 1.5583988428115845 Train_acc: 25.833334%
Train Epoch: 1/90 Train_Loss: 1.5583988428115845 Train_acc: 25.833334%
Train Epoch: 2/90 Train_Loss: 1.4648139476776123 Train_acc: 40.000004%
Train Epoch: 2/90 Train_Loss: 1.4648139476776123 Train_acc: 40.000004%
Train Epoch: 3/90 Train_Loss: 0.8719614148139954 Train_acc: 56.354168%
Train Epoch: 3/90 Train_Loss: 0.8719614148139954 Train_acc: 56.354168%
Train Epoch: 4/90 Train_Loss: 0.7760419845581055 Train_acc: 63.020836%
Train

Train Epoch: 54/90 Train_Loss: 0.06369639933109283 Train_acc: 98.750008%
Train Epoch: 55/90 Train_Loss: 0.03378120809793472 Train_acc: 98.750008%
Train Epoch: 55/90 Train_Loss: 0.03378120809793472 Train_acc: 98.750008%
Train Epoch: 56/90 Train_Loss: 0.037847381085157394 Train_acc: 98.854172%
Train Epoch: 56/90 Train_Loss: 0.037847381085157394 Train_acc: 98.854172%
Train Epoch: 57/90 Train_Loss: 0.011317435652017593 Train_acc: 98.854172%
Train Epoch: 57/90 Train_Loss: 0.011317435652017593 Train_acc: 98.854172%
Train Epoch: 58/90 Train_Loss: 0.03821415826678276 Train_acc: 98.854172%
Train Epoch: 58/90 Train_Loss: 0.03821415826678276 Train_acc: 98.854172%
Train Epoch: 59/90 Train_Loss: 0.0006923069595359266 Train_acc: 98.854172%
Train Epoch: 59/90 Train_Loss: 0.0006923069595359266 Train_acc: 98.854172%
Train Epoch: 60/90 Train_Loss: 0.01637447066605091 Train_acc: 98.958336%
Train Epoch: 60/90 Train_Loss: 0.01637447066605091 Train_acc: 98.958336%
Train Epoch: 61/90 Train_Loss: 0.0022524567

Train Epoch: 15/90 Train_Loss: 0.2538057267665863 Train_acc: 91.250008%
Train Epoch: 16/90 Train_Loss: 0.1812194585800171 Train_acc: 93.750008%
Train Epoch: 16/90 Train_Loss: 0.1812194585800171 Train_acc: 93.750008%
Train Epoch: 17/90 Train_Loss: 0.2178676575422287 Train_acc: 95.520836%
Train Epoch: 17/90 Train_Loss: 0.2178676575422287 Train_acc: 95.520836%
Train Epoch: 18/90 Train_Loss: 0.12607328593730927 Train_acc: 95.208336%
Train Epoch: 18/90 Train_Loss: 0.12607328593730927 Train_acc: 95.208336%
Train Epoch: 19/90 Train_Loss: 0.077199786901474 Train_acc: 96.562508%
Train Epoch: 19/90 Train_Loss: 0.077199786901474 Train_acc: 96.562508%
Train Epoch: 20/90 Train_Loss: 0.1826808899641037 Train_acc: 96.354172%
Train Epoch: 20/90 Train_Loss: 0.1826808899641037 Train_acc: 96.354172%
Train Epoch: 21/90 Train_Loss: 0.1816822588443756 Train_acc: 96.562508%
Train Epoch: 21/90 Train_Loss: 0.1816822588443756 Train_acc: 96.562508%
Train Epoch: 22/90 Train_Loss: 0.06227569282054901 Train_acc: 96

Train Epoch: 71/90 Train_Loss: 0.0007699838024564087 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.028696905821561813 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.028696905821561813 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.027503956109285355 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.027503956109285355 Train_acc: 98.854172%
Train Epoch: 74/90 Train_Loss: 0.0137043297290802 Train_acc: 98.854172%
Train Epoch: 74/90 Train_Loss: 0.0137043297290802 Train_acc: 98.854172%
Train Epoch: 75/90 Train_Loss: 0.07992956042289734 Train_acc: 98.854172%
Train Epoch: 75/90 Train_Loss: 0.07992956042289734 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.01056781318038702 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.01056781318038702 Train_acc: 98.854172%
Train Epoch: 77/90 Train_Loss: 0.023792428895831108 Train_acc: 98.854172%
Train Epoch: 77/90 Train_Loss: 0.023792428895831108 Train_acc: 98.854172%
Train Epoch: 78/90 Train_Loss: 0.064865760505

Train Epoch: 32/90 Train_Loss: 0.06667020171880722 Train_acc: 98.020836%
Train Epoch: 33/90 Train_Loss: 0.03896762430667877 Train_acc: 97.916672%
Train Epoch: 33/90 Train_Loss: 0.03896762430667877 Train_acc: 97.916672%
Train Epoch: 34/90 Train_Loss: 0.06881304830312729 Train_acc: 97.916672%
Train Epoch: 34/90 Train_Loss: 0.06881304830312729 Train_acc: 97.916672%
Train Epoch: 35/90 Train_Loss: 0.03939569368958473 Train_acc: 98.020836%
Train Epoch: 35/90 Train_Loss: 0.03939569368958473 Train_acc: 98.020836%
Train Epoch: 36/90 Train_Loss: 0.020165612921118736 Train_acc: 98.333336%
Train Epoch: 36/90 Train_Loss: 0.020165612921118736 Train_acc: 98.333336%
Train Epoch: 37/90 Train_Loss: 0.09861680120229721 Train_acc: 97.916672%
Train Epoch: 37/90 Train_Loss: 0.09861680120229721 Train_acc: 97.916672%
Train Epoch: 38/90 Train_Loss: 0.07720509171485901 Train_acc: 98.020836%
Train Epoch: 38/90 Train_Loss: 0.07720509171485901 Train_acc: 98.020836%
Train Epoch: 39/90 Train_Loss: 0.1503671556711197

Train Epoch: 88/90 Train_Loss: 0.008956658653914928 Train_acc: 98.645836%
Train Epoch: 89/90 Train_Loss: 0.01736702024936676 Train_acc: 98.645836%
Train Epoch: 89/90 Train_Loss: 0.01736702024936676 Train_acc: 98.645836%
Train Epoch: 90/90 Train_Loss: 0.05511116608977318 Train_acc: 98.645836%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 88.750000%
Training Time: 62.41 seconds
Train Epoch: 90/90 Train_Loss: 0.05511116608977318 Train_acc: 98.645836%
Test accuracy of the model: 69.250000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 88.750000%
Training Time: 62.41 seconds
----------------------------------------------------------------------------------------------------------------
fold:  4
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
---------------------------------------------------------------------------------------------

Train Epoch: 49/90 Train_Loss: 0.013669449836015701 Train_acc: 98.645836%
Train Epoch: 50/90 Train_Loss: 0.013530229218304157 Train_acc: 98.541672%
Train Epoch: 50/90 Train_Loss: 0.013530229218304157 Train_acc: 98.541672%
Train Epoch: 51/90 Train_Loss: 0.004120709840208292 Train_acc: 98.645836%
Train Epoch: 51/90 Train_Loss: 0.004120709840208292 Train_acc: 98.645836%
Train Epoch: 52/90 Train_Loss: 0.03919436037540436 Train_acc: 98.437508%
Train Epoch: 52/90 Train_Loss: 0.03919436037540436 Train_acc: 98.437508%
Train Epoch: 53/90 Train_Loss: 0.012371348217129707 Train_acc: 98.645836%
Train Epoch: 53/90 Train_Loss: 0.012371348217129707 Train_acc: 98.645836%
Train Epoch: 54/90 Train_Loss: 0.031859688460826874 Train_acc: 98.645836%
Train Epoch: 54/90 Train_Loss: 0.031859688460826874 Train_acc: 98.645836%
Train Epoch: 55/90 Train_Loss: 0.040763307362794876 Train_acc: 98.541672%
Train Epoch: 55/90 Train_Loss: 0.040763307362794876 Train_acc: 98.541672%
Train Epoch: 56/90 Train_Loss: 0.0408994

Train Epoch: 10/90 Train_Loss: 0.6357279419898987 Train_acc: 81.354172%
Train Epoch: 11/90 Train_Loss: 0.39352303743362427 Train_acc: 81.979172%
Train Epoch: 11/90 Train_Loss: 0.39352303743362427 Train_acc: 81.979172%
Train Epoch: 12/90 Train_Loss: 0.5604934692382812 Train_acc: 86.562508%
Train Epoch: 12/90 Train_Loss: 0.5604934692382812 Train_acc: 86.562508%
Train Epoch: 13/90 Train_Loss: 0.42145270109176636 Train_acc: 87.395836%
Train Epoch: 13/90 Train_Loss: 0.42145270109176636 Train_acc: 87.395836%
Train Epoch: 14/90 Train_Loss: 0.22356188297271729 Train_acc: 90.104172%
Train Epoch: 14/90 Train_Loss: 0.22356188297271729 Train_acc: 90.104172%
Train Epoch: 15/90 Train_Loss: 0.2509731352329254 Train_acc: 89.895836%
Train Epoch: 15/90 Train_Loss: 0.2509731352329254 Train_acc: 89.895836%
Train Epoch: 16/90 Train_Loss: 0.12986987829208374 Train_acc: 91.145836%
Train Epoch: 16/90 Train_Loss: 0.12986987829208374 Train_acc: 91.145836%
Train Epoch: 17/90 Train_Loss: 0.18766272068023682 Train

Train Epoch: 67/90 Train_Loss: 0.1533225178718567 Train_acc: 98.437508%
Train Epoch: 67/90 Train_Loss: 0.1533225178718567 Train_acc: 98.437508%
Train Epoch: 68/90 Train_Loss: 0.09453895688056946 Train_acc: 98.437508%
Train Epoch: 68/90 Train_Loss: 0.09453895688056946 Train_acc: 98.437508%
Train Epoch: 69/90 Train_Loss: 0.027764342725276947 Train_acc: 98.645836%
Train Epoch: 69/90 Train_Loss: 0.027764342725276947 Train_acc: 98.645836%
Train Epoch: 70/90 Train_Loss: 0.013148106634616852 Train_acc: 98.541672%
Train Epoch: 70/90 Train_Loss: 0.013148106634616852 Train_acc: 98.541672%
Train Epoch: 71/90 Train_Loss: 0.023700037971138954 Train_acc: 98.541672%
Train Epoch: 71/90 Train_Loss: 0.023700037971138954 Train_acc: 98.541672%
Train Epoch: 72/90 Train_Loss: 0.048974569886922836 Train_acc: 98.541672%
Train Epoch: 72/90 Train_Loss: 0.048974569886922836 Train_acc: 98.541672%
Train Epoch: 73/90 Train_Loss: 0.15602362155914307 Train_acc: 98.541672%
Train Epoch: 73/90 Train_Loss: 0.156023621559

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()